In [21]:
import os
from keras import layers, models
from keras import optimizers,losses
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import gc

In [22]:
# import tensorflow as tf
# tf.config.list_physical_devices()
#
# import keras.backend as k
#k.clear_session()

In [23]:
orig_data_dir = 'D:\Deep-learning\\test-scripts\dogs-vs-cats\\train'
bas_dir = 'D:\Deep-learning\\test-scripts'

train_dir = os.path.join(bas_dir,'train')
print(train_dir)

val_dir = os.path.join(bas_dir,'val')
print(val_dir)

test_dir = os.path.join(bas_dir,'test')
print(test_dir)




D:\Deep-learning\test-scripts\train
D:\Deep-learning\test-scripts\val
D:\Deep-learning\test-scripts\test


In [24]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150,150), batch_size=20, class_mode='binary')
val_generator = test_datagen.flow_from_directory(val_dir,target_size=(150,150), batch_size=20, class_mode='binary')
#test_generator = test_datagen.flow_from_directory(test_dir,target_size==(150,150), batch_size=20, class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [25]:
for data_batch ,labels_batch in train_generator:
    print('Data Batch Shape', data_batch.shape)
    print('Labels Shape', labels_batch.shape)
    break


Data Batch Shape (20, 150, 150, 3)
Labels Shape (20,)


Defining the basic model


In [26]:
model_basic = models.Sequential()

model_basic.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model_basic.add(layers.MaxPooling2D((2,2)))
model_basic.add(layers.Conv2D(64,(3,3),activation='relu'))
model_basic.add(layers.MaxPooling2D((2,2)))
model_basic.add(layers.Conv2D(128,(3,3),activation='relu'))
model_basic.add(layers.MaxPooling2D((2,2)))
model_basic.add(layers.Conv2D(128,(3,3),activation='relu'))
model_basic.add(layers.MaxPooling2D((2,2)))
model_basic.add(layers.Flatten())
model_basic.add(layers.Dense(512,activation='relu'))
model_basic.add(layers.Dense(1,activation='sigmoid'))


In [27]:
model_basic.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

Setting the loss and optimizer settings for the model to be trained.

In [28]:
model_basic.compile(loss='binary_crossentropy',optimizer = optimizers.RMSprop(lr=1e-4),metrics=['acc'])


Training the model on the 2000 training images for 30 epochs and tracking all the accs and losses

In [29]:
history = model_basic.fit_generator(train_generator,steps_per_epoch=100,epochs=30,validation_data=val_generator,validation_steps=50)

Epoch 1/30
100/100 [==============================] - 5s 55ms/step - loss: 0.6855 - acc: 0.5495 - val_loss: 0.7000 - val_acc: 0.5190

KeyboardInterrupt: 

In [ ]:
## Saving the model locally
model_basic.save('Kaggle-cats-dogs-small-basic.h5')


Plotting the Accuracies and Losses over Training and Validation Dataset

In [ ]:
acc  = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)
plt.plot(epochs,acc,'b-',label='Training Acc')
plt.plot(epochs,val_acc,'r-',label='Validation Acc')
plt.title('Training and Validation Accuracies')
plt.legend()
plt.figure()

epochs = range(1,len(acc)+1)
plt.plot(epochs,loss,'b-',label='Training Loss')
plt.plot(epochs,val_loss,'r-',label='Validation Loss')
plt.title('Training and Validation losses')
plt.legend()
plt.figure()

del history
del model_basic
gc.collect()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import keras.backend as k
k.clear_session()

Data Augmentation to overcome overfitting




In [ ]:
datagen = ImageDataGenerator(rotation_range=40,width_shift_range=0.2,height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')

Defining a new model with dropout to overcome over-fitting

In [ ]:
model_dropout = models.Sequential()
model_dropout.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model_dropout.add(layers.MaxPooling2D((2,2)))
model_dropout.add(layers.Conv2D(64,(3,3),activation='relu'))
model_dropout.add(layers.MaxPooling2D((2,2)))
model_dropout.add(layers.Conv2D(128,(3,3),activation='relu'))
model_dropout.add(layers.MaxPooling2D((2,2)))
model_dropout.add(layers.Conv2D(128,(3,3),activation='relu'))
model_dropout.add(layers.MaxPooling2D((2,2)))
model_dropout.add(layers.Flatten())
model_dropout.add(layers.Dropout(0.5))
model_dropout.add(layers.Dense(512,activation='relu'))
model_dropout.add(layers.Dense(1,activation='sigmoid'))

model_dropout.summary()

In [ ]:
model_dropout.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-3),metrics=['acc'])


Defining the new training data generator using data augmentation techniques and keeping the validation generator the same.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150,150),batch_size=20,class_mode='binary')
val_generator = test_datagen.flow_from_directory(val_dir,target_size=(150,150),batch_size=20,class_mode='binary')

Training the new dropout model for 100 epochs

In [ ]:
history = model_dropout.fit(train_generator,steps_per_epoch=100,epochs=50,validation_data=val_generator,validation_steps=50)

In [ ]:
model_dropout.save('Kaggle-cats-dogs-small-dropout.h5')

In [ ]:
acc  = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)
plt.plot(epochs,acc,'b-',label='Training Acc')
plt.plot(epochs,val_acc,'r-',label='Validation Acc')
plt.title('Training and Validation Accuracies')
plt.legend()
plt.figure()

epochs = range(1,len(acc)+1)
plt.plot(epochs,loss,'b-',label='Training Loss')
plt.plot(epochs,val_loss,'r-',label='Validation Loss')
plt.title('Training and Validation losses')
plt.legend()
plt.figure()

del history
del model_basic
gc.collect()
k.clear_session()
